# Tinder automated like
This app allows you to send Likes on tinder automatically.

facebook_token, which expires almost in an hour, can be got in the url after pusshing OK button in https://www.facebook.com/dialog/oauth?client_id=464891386855067&redirect_uri=fbconnect://success&scope=basic_info%2Cemail%2Cpublic_profile%2Cuser_about_me%2Cuser_activities%2Cuser_birthday%2Cuser_education_history%2Cuser_friends%2Cuser_interests%2Cuser_likes%2Cuser_location%2Cuser_photos%2Cuser_relationship_details&response_type=token&__mref=message

facebook_id can be found from http://findmyfbid.com

In [25]:
import requests
import pynder
import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
import sqlite3
import time

In [99]:
facebook_id = 'your_id'
facebook_auth_token = 'your_token'

In [100]:
session = pynder.Session(facebook_id, facebook_auth_token)

In [ ]:
session.matches() # get users you have already been matched with

In [106]:
LAT = 35.6804
LON = 139.7717

In [107]:
session.update_location(LAT, LON) # updates latitude and longitude for your profile

{'status': 200}

In [74]:
profile = session.profile  # your profile. If you update its attributes they will be updated on Tinder.

In [108]:
conn = sqlite3.connect('tinder.db')
conn.isolation_level = None
c = conn.cursor()

In [109]:
c.execute('''CREATE TABLE IF NOT EXISTS girls
             (id text primary key, name text, bio text, age integer, birth_date timestamp,
             ping_time text, distance_km real, like integer, match integer)''')
c.execute('''CREATE TABLE IF NOT EXISTS photos
             (id text, photo_url text, scraped_date date, unique(id, photo_url))''')
c.execute('''CREATE TABLE IF NOT EXISTS schools
             (id text, school text, scraped_date date, unique(id, school))''')
c.execute('''CREATE TABLE IF NOT EXISTS jobs
             (id text, job text, scraped_date date, unique(id, job))''')
c.execute('''CREATE TABLE IF NOT EXISTS common_connections
             (id text, friend text, scraped_date date, unique(id, friend))''')
c.execute('''CREATE TABLE IF NOT EXISTS instagram_username
             (id text, username text, scraped_date date, unique(id, username))''')

In [114]:
cnt = 0
today = datetime.datetime.today()
while session.can_like_in < 0:
    users = session.nearby_users() # returns a list of users nearby
    girls = [(user.id, user.name, user.bio, user.age, user.birth_date, datetime.datetime.strptime(user.ping_time, '%Y-%m-%dT%H:%M:%S.%fZ'),
             user.distance_km, 1, 0) for user in users]
    photos = [(user.id, photo, today) for user in users for photo in user.get_photos(width='640') if user.photos] # (id, photo_url)
    schools = [(user.id, school, today) for user in users for school in user.schools] # (id, school)
    jobs = [(user.id, job, today) for user in users for job in user.jobs] # (id, job)
    common_connections = [(user.id, conn, today) for user in users for conn in user.common_connections] # (id, friend)
    instagram_username = [(user.id, user.instagram_username, today) for user in users if user.instagram_username] # (id, username)
    try:
        c.execute('begin')
        c.executemany('INSERT OR REPLACE INTO girls VALUES (?,?,?,?,?,?,?,?,?)', girls)
        c.executemany('INSERT OR IGNORE INTO photos VALUES (?,?,?)', photos)
        c.executemany('INSERT OR IGNORE INTO schools VALUES (?,?,?)', schools)    
        c.executemany('INSERT OR IGNORE INTO jobs VALUES (?,?,?)', jobs)
        c.executemany('INSERT OR IGNORE INTO common_connections VALUES (?,?,?)', common_connections)
        c.executemany('INSERT OR IGNORE INTO instagram_username VALUES (?,?,?)', instagram_username)
        c.execute('end')
    except:
        c.execute('rollback')
    matches = [user.superlike() if 'University of Tokyo' in user.schools or '東京大学' in user.schools else user.like() for user in users]
    cnt += len(users)
    time.sleep(0.5)
    print('You liked {} girls.'.format(cnt))
nexttime = datetime.datetime.now() + datetime.timedelta(seconds=session.can_like_in)
print('You can like again at '+nexttime.strftime("%Y-%m-%d %H:%M:%S"))

You liked 11 girls.
You liked 22 girls.
You liked 33 girls.
You liked 44 girls.
You liked 55 girls.
You can like again at 2017-03-17 04:51:50


In [ ]:
conn.commit()
conn.close()